# Using Twitter API

In [ ]:
import requests 
import pandas as pd 

with open('bearer_token.txt') as fp:
    BEARER_TOKEN = fp.read()

In [1]:
# https://api.twitter.com/1.1
# https://api.twitter.com/2

In [ ]:
# we will be using v2

API = https://api.twitter.com/2 <br>
Endpoint = search/tweets.json <br>
Query = query=Tesla

In [ ]:
response = requests.get('https://api.twitter.com/2/tweets/search/recent?query=Tesla')
response.json()

In [2]:
headers = {'authorization': f'Bearer {BEARER_TOKEN}'}
response = requests.get('https://api.twitter.com/2/tweets/search/recent?query=Tesla',
                       headers=headers)
response.json()

NameError: name 'BEARER_TOKEN' is not defined

In [ ]:
headers = {'authorization': f'Bearer {BEARER_TOKEN}'}
response = requests.get('https://api.twitter.com/2/tweets/search/recent?query=Tesla&tweet.fields=created_at,lang&max_results=100',
                       headers=headers)
response.json()

In [ ]:
headers = {'authorization': f'Bearer {BEARER_TOKEN}'}
params = {
    'query': 'Tesla',
    'tweet.fields': 'created_at,lang',
    'max_results':100
}
response = requests.get('https://api.twitter.com/2/tweets/search/recent',
                       headers=headers, params=params)
response.json()

In [ ]:
str(response.json())[:500]

In [ ]:
response.json()['data']

In [ ]:
response.json()['data'][0]

In [ ]:
response.json()['data'][0]['text']

In [ ]:
tweet = response.json()['data'][0]

pd.DataFrame = ({
    'id': [tweet['id']],
    'text': [tweet['text']],
    'lang': [tweet['lang']],
    'created_at': [tweet['created_at']]
})

In [ ]:
df = pd.DataFrame()
for tweet in response.json()['data']:
    df = df.append(tweet, ignore_index=True)
    
df.head()

# Twitter data collection 

In [ ]:
import requests 
import pandas as pd 
from datetime import datetime, timedelta 

with open('bearer_token.txt') as fp:
    BEARER_TOKEN = fp.read()

In [ ]:
endpoint = "https://api.twitter.com/2/tweets/search/recent"
headers = {'authorization': f'Bearer {BEARER_TOKEN}'}
params = {
    'query': 'tesla',
    'max_results': '100',
    'tweet.fields': 'created_at, lang'
}

In [ ]:
params['query'] = '(tesla OR tsla OR elon musk) (lang:en)'

In [ ]:
datetime.now()

In [ ]:
time = datetime.now()
print(time.hour)

In [ ]:
timedelta(hours=3)

In [ ]:
time.strftime('%d/%m/%Y <- date | time -> %H:%M:%S')

In [ ]:
time.strftime("%Y-%m-%dT%H:%M:%SZ")

In [ ]:
for hour in range(24):
    print(f'-{hour}')
    pre60 = time - timedelta(minutes=60)
    print(f'end\t{time.strftime("%Y-%m-%dT%H:%M:%SZ")}')
    print(f'start\t{pre60.strftime("%Y-%m-%dT%H:%M:%SZ")}')
    time = pre60
    print()

In [ ]:
dtformat = "%Y-%m-%dT%H:%M:%SZ"

In [ ]:
df = pd.DataFrame()

for hour in range(24*7 - 1):
    pre60 = time - timedelta(minutes=60)
    
    params['end_time'] = time.strftime(dtformat)
    params['start_time'] = pre60.strftime(dtformat)
    
    response = requests.get(endpoint, headers=headers, params=params)
    
    time = pre60
    
    for tweet in response.json()['data']:
        df = df.append(tweet, ignore_index=True)
        
    print(params['end_time'])

In [ ]:
response.json()

In [ ]:
df['created_at'].min()

In [ ]:
df['created_at'].max()

In [ ]:
df.replace('|', '', inplace=True)

In [ ]:
df.to_csv('tesla_tweets.csv', sep='|', index=False)

# Sentiment Analysis 

In [ ]:
!pip install flair
!pip install tensorflow
!pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

Loading DistilBERT model

In [ ]:
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

In [ ]:
tweets = pd.read_csv('tesla_tweets.csv', sep='|')
len(tweets)

In [ ]:
sentence = flair.data.Sentence(tweets['text'].iloc[0])
sentence

In [ ]:
sentiment_model.predict(sentence)

In [ ]:
sentence 

In [ ]:
help(sentence)

In [ ]:
sentence.get_labels()[0]

In [ ]:
help(sentence.get_labels()[0])

In [ ]:
sentence.get_labels()[0].score

In [ ]:
sentence.get_labels()[0].value

In [ ]:
sentence.labels[0].value, sentence.labels[0].score

### Results 

In [ ]:
sentiment = []
confidence = []

sample = tweets.iloc[:1000]

for tweet in sample['text'].to_list():
    sentence = flair.data.Sentence(tweet)
    sentiment_model.predict(sentence)
    
    sentiment.append(sentence.labels[0].value)
    confidence.append(sentence.labels[0].score)
    
sample['sentiment'] = sentiment
sample['confidence'] = confidence

sample.head()

In [ ]:
pd.set_option('display.max_colwidth', None)
sample.head()